# 1) Datasets

Vamos a leer el dataset que envía PyP

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('D:/MOPC/Enriquecimiento de Datos/Data Cleaning/Active Historico/Datasets/active_historico.csv')

In [3]:
df.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO
0,DNI 22602516,LEGUIZAMON RAUL ALBERTO,45470245,TELEFONO,AH
1,DNI 5655203,ROMERO TERESITA MARIA DEL CARM,0001147374189,TELEFONO,AH
2,DNI 29951056,AIRA NOEMI INES,39688310,TELEFONO,AH
3,DNI 18004317,ROLDAN LUIS ANIBAL,0001142953815,TELEFONO,AH
4,COL 12142024,OBREGON EDGAR,0000000000,TELEFONO,AH


In [4]:
df.shape

(484628, 5)

## 1.2) Data Cleaning: Datasets

Vamos a dropear aquellas columnas y filas que no tengan ningún valor (es decir, que sean todos NaNs)

In [5]:
df = df.dropna(axis=0, how='all')

In [6]:
df = df.dropna(axis=1, how='all')

In [7]:
df.shape

(484628, 5)

Ahora todas las columnas tienen al menos 1 dato.

# 2) Cleaning: Teléfonos

Verificar teléfonos if DF TIPO is TELEFONO

In [8]:
df_telefonos = df[df['TIPO']== 'TELEFONO']

In [9]:
df_telefonos.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO
0,DNI 22602516,LEGUIZAMON RAUL ALBERTO,45470245,TELEFONO,AH
1,DNI 5655203,ROMERO TERESITA MARIA DEL CARM,0001147374189,TELEFONO,AH
2,DNI 29951056,AIRA NOEMI INES,39688310,TELEFONO,AH
3,DNI 18004317,ROLDAN LUIS ANIBAL,0001142953815,TELEFONO,AH
4,COL 12142024,OBREGON EDGAR,0000000000,TELEFONO,AH


In [10]:
df_telefonos.shape

(472713, 5)

In [11]:
df_telefonos['DATO'].value_counts().head(50)

000                               28178
0000000000                        25588
00000000000                       15707
00                                 2965
TT                                 1313
EQUIV                              1016
TT OK                               705
-                                   352
43188500                            309
N                                   305
EQUIVOCADO                          289
equiv                               276
NCAS                                267
LAB                                 255
NCCS                                227
tt                                  224
MADRE                               202
equivocado                          190
03364460380                         183
VER                                 162
VECINA                              162
000000000000                        153
FLIAR                               150
CEL TT                              134
45763300                            105


Como regla incluso a nivel de pipeline: **todo lo que aparezca más de 10 veces es basura**.

In [12]:
df_telefonos['DATO_CLEAN'] = [re.sub("[^0-9]", "", str(df_telefonos['DATO'][i])) for i in range (df_telefonos['DATO'].count())]

<ipython-input-12-971e25ea3f5b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_telefonos['DATO_CLEAN'] = [re.sub("[^0-9]", "", str(df_telefonos['DATO'][i])) for i in range (df_telefonos['DATO'].count())]


In [13]:
df_telefonos['DATO_CLEAN'].value_counts().head(50)

000             28256
0000000000      25588
                19771
00000000000     15707
00               2977
43188500          429
03364460380       248
000000000000      153
45763300          148
18                128
43633100          127
19                121
49598200          118
43481200          107
43464133          107
43401100          106
0340478200        105
14                 98
02901433460        96
15                 91
40037000           91
17                 88
20                 87
02214296100        87
59831420           82
47569000           77
48578118           77
16                 75
46306600           73
03364438100        72
48304100           72
0000               71
47112000           70
47043000           69
47098000           68
13                 67
49596100           63
44320606           60
45092555           59
56272000           57
02214210808        56
43292000           52
03442441800        51
02214298300        51
03444425757        48
0344742218

## Condiciones

In [14]:
#df_telefonos.loc[((df_telefonos['DATO_CLEAN'].str.len() >= 7)) , 'TELEFONO'] = df_telefonos['DATO_CLEAN']

In [15]:
df_telefonos.tail()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,DATO_CLEAN
472708,BBVA16543219,VELEZ IRMA,00,TELEFONO,AH,00
472709,COL 12247453,DIAZ FRANCISCO DAVID,FLIAR D TT,TELEFONO,AH,
472710,DNI 18012467,MARTINEZ CLAUDIO HECTOR,02657433104,TELEFONO,AH,02657433104
472711,COL 28883717,PEREZ MANUEL DELFIN,8 a 12 hs / 16 a 20,TELEFONO,AH,8121620
472712,COL 11722100,GARCIA CARLOS ALBERTO,TT OK,TELEFONO,AH,


Luego, que tenga al menos 2 caracteres diferentes. Por lo que nos quedamos con aquellos que tengan una len(set(str)) > 1

In [16]:
a = '88888888888'

In [17]:
len (set(a))

1

Defino una función para chequear los caracteres diferentes

In [18]:
def distinct_character(string):
    dist_chars = len(set(string))
    if dist_chars > 1:
        ans = True
    else:
        ans = False
    return ans

Creo una columna temporal donde van los True/False

In [19]:
df_telefonos['DIST_CHARACTER'] = df_telefonos['DATO_CLEAN'].apply(distinct_character) 

<ipython-input-19-c16dd8c93c53>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_telefonos['DIST_CHARACTER'] = df_telefonos['DATO_CLEAN'].apply(distinct_character)


In [20]:
df_telefonos.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,DATO_CLEAN,DIST_CHARACTER
0,DNI 22602516,LEGUIZAMON RAUL ALBERTO,45470245,TELEFONO,AH,45470245,True
1,DNI 5655203,ROMERO TERESITA MARIA DEL CARM,0001147374189,TELEFONO,AH,0001147374189,True
2,DNI 29951056,AIRA NOEMI INES,39688310,TELEFONO,AH,39688310,True
3,DNI 18004317,ROLDAN LUIS ANIBAL,0001142953815,TELEFONO,AH,0001142953815,True
4,COL 12142024,OBREGON EDGAR,0000000000,TELEFONO,AH,0000000000,False


In [21]:
df_telefonos.loc[((df_telefonos['DIST_CHARACTER'] == True) & (df_telefonos['DATO_CLEAN'].str.len() >= 7)
            ) , 'TELEFONO'] = df_telefonos['DATO_CLEAN']

C:\Users\Dleg\anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Dleg\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [22]:
df_telefonos.loc[((df_telefonos['DIST_CHARACTER'] == False) & (df_telefonos['DATO_CLEAN'].str.len() < 7)
                 ) , 'TELEFONO'] = np.nan

In [23]:
df_telefonos.head()

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,DATO_CLEAN,DIST_CHARACTER,TELEFONO
0,DNI 22602516,LEGUIZAMON RAUL ALBERTO,45470245,TELEFONO,AH,45470245,True,45470245
1,DNI 5655203,ROMERO TERESITA MARIA DEL CARM,0001147374189,TELEFONO,AH,0001147374189,True,0001147374189
2,DNI 29951056,AIRA NOEMI INES,39688310,TELEFONO,AH,39688310,True,39688310
3,DNI 18004317,ROLDAN LUIS ANIBAL,0001142953815,TELEFONO,AH,0001142953815,True,0001142953815
4,COL 12142024,OBREGON EDGAR,0000000000,TELEFONO,AH,0000000000,False,NaN


In [24]:
df_telefonos['TELEFONO'].value_counts().head(30)

43188500       429
03364460380    248
45763300       148
43633100       127
49598200       118
43481200       107
43464133       107
43401100       106
0340478200     105
02901433460     96
40037000        91
02214296100     87
59831420        82
47569000        77
48578118        77
46306600        73
03364438100     72
48304100        72
47112000        70
47043000        69
47098000        68
49596100        63
44320606        60
45092555        59
56272000        57
02214210808     56
43292000        52
03442441800     51
02214298300     51
03444425757     48
Name: TELEFONO, dtype: int64

In [25]:
df_telefonos[df_telefonos['IDMOROSO'] == 'COL 13747801      ']

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,DATO_CLEAN,DIST_CHARACTER,TELEFONO
91982,COL 13747801,LAZO PABLO WALTER,0000000000,TELEFONO,AH,0000000000,False,NaN
266411,COL 13747801,LAZO PABLO WALTER,<< ANTES D 19 HS,TELEFONO,AH,19,True,NaN
358924,COL 13747801,LAZO PABLO WALTER,02614238259,TELEFONO,AH,02614238259,True,02614238259
465727,COL 13747801,LAZO PABLO WALTER,TRABAJO,TELEFONO,AH,,False,NaN


## Drop

Dropeamos aquellos que tengan NaN en la columna Telefono

In [26]:
df_telefonos = df_telefonos.dropna(subset=['TELEFONO'])

In [27]:
df_telefonos[df_telefonos['IDMOROSO'] == 'COL 13747801      ']

,IDMOROSO,NOMBRE,DATO,TIPO,PROVEEDOR_DATO,DATO_CLEAN,DIST_CHARACTER,TELEFONO
358924,COL 13747801,LAZO PABLO WALTER,02614238259,TELEFONO,AH,02614238259,True,02614238259


In [28]:
df_telefonos.shape

(375338, 8)

In [29]:
df_telefonos = df_telefonos [['IDMOROSO','TELEFONO','TIPO','PROVEEDOR_DATO']]

## 12.2) DNI Cleaning

Limpiamos el dataframe __df_pyp__, ya que dentro de DATO_A_ENVIAR tenemos 9 tipos de casos diferentes de datos:

- DNI 12345678
- DNI12345678
- DNIE1234678
- LC  1234567
- LE  1234567
- CUIT00123456780
- 00123456780 --> Es un CUIT.
- 00 12345678 0 --> Es un CUIT.
- DNI 00123456780 --> Dice DNI pero es un CUIT de una persona __jurídica__ (TO DO)

Para eso definimos 5 funciones:

- __Tipo_DOC__: Se definen dos funciones para determinar el tipo de DNI. Una inicial, y otra para corregir sobre el dataframe resultante.
- __Standard DNI__: Estandariza los DNI para 7 de los casos. Esta función debe ejecutarse en primer lugar.
- __DNI Spaces__: Elimina los espacios.
- __DNI Zero__: Elimina los ceros iniciales de ciertos CUITs para que coincidan con los DNI.
- __Cuit__: Decisión sobre dejar un cuit o volverlo DNI

In [30]:
def tipo_doc_1 (string_dni):
    
    if 'DNI' in string_dni:
        tipo = 'DNI'
    
    elif 'LC' in string_dni:
        tipo = 'LC'
    
    elif 'LE' in string_dni:
        tipo = 'LE'
    
    elif 'DNIE' in string_dni:
        tipo = 'DNIE'    
    
    elif ((string_dni.startswith(('CUIT30','CUIT33','CUIT34'))) & (len(string_dni)>8)):
        tipo = 'CUIT'
        
    elif ((string_dni.startswith(('30','33','34'))) & (len(string_dni)>8)):
        tipo = 'CUIT'

    elif ((string_dni.startswith(('20','23','24','27'))) & (len(string_dni)>8)):
        tipo = 'DNI'

    elif ((string_dni.startswith(('CUIT20','CUIT23','CUIT24','CUIT27'))) & (len(string_dni)>8)):
        tipo = 'DNI'
        
    else:
        tipo = 'DNI'
    
    return tipo

In [31]:
def tipo_doc_2 (string_dni, tipo_original):
    
    if (len(string_dni) > 8) & (tipo_original == 'DNI'):
        tipo = 'CUIT'
    else:
        tipo = tipo_original
    return tipo

In [32]:
def standard_dni (string_dni):
    """DNI separation by each case.
    """
    
    if 'DNI ' in string_dni:
        dni = string_dni.split('DNI ')[1]
        
    elif 'DNIE' in string_dni:
        dni = string_dni.split('DNIE')[1]
    
    elif 'DNI' in string_dni:
        dni = string_dni.split('DNI')[1]
    
    elif 'LC  ' in string_dni:
        dni = string_dni.split('LC  ')[1]
    
    elif 'LE  ' in string_dni:
        dni = string_dni.split('LE  ')[1]
    
    elif 'CUIT' in string_dni:
        dni = string_dni.split('CUIT')[1]
    
    elif 'COL ' in string_dni:
        dni = string_dni.split('COL ')[1]
        
    elif 'BBVA' in string_dni:
        dni = string_dni.split('BBVA')[1]
                
        
    else:
        dni = string_dni
        
    return dni

In [33]:
def dni_characters (string_dni):
    
    if ' ' in string_dni:
        dni = string_dni.replace(' ','')
        
    elif '-' in string_dni:
        dni = string_dni.replace('-','')

    else:
        dni = string_dni
    
    return dni

In [34]:
def cuit (string_dni):
    
    if ((string_dni.startswith('20')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]
        
    elif ((string_dni.startswith('23')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]
        
    elif ((string_dni.startswith('24')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]

    elif ((string_dni.startswith('27')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]

    else:
        cuit = string_dni
        
    return cuit

In [35]:
def dni_zero (string_dni):
    
    if string_dni.startswith('0'):
        dni = string_dni [1:]

    else:
        dni = string_dni
    
    return dni

En primer lugar, trimmeamos los espacios que sobran:

In [36]:
df_telefonos

,IDMOROSO,TELEFONO,TIPO,PROVEEDOR_DATO
0,DNI 22602516,45470245,TELEFONO,AH
1,DNI 5655203,0001147374189,TELEFONO,AH
2,DNI 29951056,39688310,TELEFONO,AH
3,DNI 18004317,0001142953815,TELEFONO,AH
5,COL 12454241,43077577,TELEFONO,AH
...,...,...,...,...
472697,DNI 10025321,03624432089,TELEFONO,AH
472699,COL 13289245,03722427803,TELEFONO,AH
472706,DNI 14216939,42673883,TELEFONO,AH
472710,DNI 18012467,02657433104,TELEFONO,AH


In [37]:
df_telefonos['NRO_DOC'] = df_telefonos['IDMOROSO'].str.strip()

Se implementan estas funciones:

Para determinar el __TIPO DE DOCUMENTO__ en primer lugar:

In [38]:
df_telefonos['TIPO_DOC'] = df_telefonos['NRO_DOC'].apply(tipo_doc_1)

Para la limpieza de DNIs:

In [39]:
df_telefonos['NRO_DOC'] = df_telefonos['NRO_DOC'].apply(standard_dni)

In [40]:
df_telefonos['NRO_DOC'] = df_telefonos['NRO_DOC'].apply(dni_characters)

In [41]:
df_telefonos['NRO_DOC'] = df_telefonos['NRO_DOC'].apply(cuit)

In [42]:
df_telefonos['NRO_DOC'] = df_telefonos['NRO_DOC'].apply(dni_zero)

Para corregir el __TIPO DE DOCUMENTO__ sobre el dataframe resultante

In [43]:
# REVISAR

#df_telefonos['TIPO_DOC'] = df_telefonos['NRO_DOC'].apply(tipo_doc_2(df_telefonos['NRO_DOC'],df_telefonos['TIPO_DOC']))

In [44]:
df_telefonos.head()

,IDMOROSO,TELEFONO,TIPO,PROVEEDOR_DATO,NRO_DOC,TIPO_DOC
0,DNI 22602516,45470245,TELEFONO,AH,22602516,DNI
1,DNI 5655203,0001147374189,TELEFONO,AH,5655203,DNI
2,DNI 29951056,39688310,TELEFONO,AH,29951056,DNI
3,DNI 18004317,0001142953815,TELEFONO,AH,18004317,DNI
5,COL 12454241,43077577,TELEFONO,AH,12454241,DNI


In [45]:
#df_telefonos.to_csv('ah_test_tel.csv')

## Campos adicionales

Asigno campos adicionales que no tengo

In [46]:
df_telefonos = df_telefonos.assign(CUIT_CUIL=np.nan)

In [47]:
df_telefonos = df_telefonos.assign(SUBTIPO='PERSONAL')

In [48]:
df_telefonos.loc[((df_telefonos['TELEFONO'].str.startswith('11')) |((df_telefonos['TELEFONO'].str.startswith('15'))
            
            )) , 'TIPO'] = 'TEL.CELULAR'

In [49]:
df_telefonos.loc[(df_telefonos['TIPO'] != 'TEL.CELULAR') , 'TIPO'] = 'TEL.FIJO'

In [50]:
df_telefonos = df_telefonos.rename(columns = {'TELEFONO':'DATO'})

In [51]:
df_telefonos = df_telefonos.assign(PROPIETARIO_DATO='Propio')

In [52]:
df_telefonos = df_telefonos.assign(FECHA_INGRESO_DATO=pd.to_datetime('today'))

In [53]:
df_telefonos['VERIFICADO'] = False

In [54]:
df_telefonos['FECHA_VERIFICACION'] = np.nan

In [55]:
df_telefonos['OPERADOR_VERIFICACION'] = np.nan

In [56]:
df_telefonos['SCORE_DATO'] = 1.5

In [57]:
df_telefonos.head()

,IDMOROSO,DATO,TIPO,PROVEEDOR_DATO,NRO_DOC,TIPO_DOC,CUIT_CUIL,SUBTIPO,PROPIETARIO_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO
0,DNI 22602516,45470245,TEL.FIJO,AH,22602516,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
1,DNI 5655203,0001147374189,TEL.FIJO,AH,5655203,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
2,DNI 29951056,39688310,TEL.FIJO,AH,29951056,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
3,DNI 18004317,0001142953815,TEL.FIJO,AH,18004317,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
5,COL 12454241,43077577,TEL.FIJO,AH,12454241,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5


# Export

In [58]:
df_telefonos.shape

(375338, 14)

In [59]:
duplicateRowsDF = df_telefonos[df_telefonos.duplicated(keep='first')]

In [60]:
duplicateRowsDF

,IDMOROSO,DATO,TIPO,PROVEEDOR_DATO,NRO_DOC,TIPO_DOC,CUIT_CUIL,SUBTIPO,PROPIETARIO_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO
11,DNI 24382483,02614377888,TEL.FIJO,AH,24382483,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
72,DNI 18264106,03414381337,TEL.FIJO,AH,18264106,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
77,DNI 18120320,44578130,TEL.FIJO,AH,18120320,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
84,DNI 17896825,47717643,TEL.FIJO,AH,17896825,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
158,DNI 26399277,2322420441,TEL.FIJO,AH,26399277,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472626,DNI 17488995,41292400,TEL.FIJO,AH,17488995,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
472630,DNI 17252470,02614215489,TEL.FIJO,AH,17252470,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
472642,DNI 17185230,46511306,TEL.FIJO,AH,17185230,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
472643,DNI 17185230,46511306,TEL.FIJO,AH,17185230,DNI,NaN,PERSONAL,Propio,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5


In [61]:
duplicateRowsDF.shape

(13277, 14)

In [62]:
df_export = df_telefonos.drop_duplicates()

In [63]:
df_export.shape

(362061, 14)

In [64]:
df_export = df_export[['IDMOROSO', 'TIPO_DOC','NRO_DOC','CUIT_CUIL','TIPO','SUBTIPO','DATO','PROPIETARIO_DATO','PROVEEDOR_DATO','FECHA_INGRESO_DATO','VERIFICADO','FECHA_VERIFICACION','OPERADOR_VERIFICACION','SCORE_DATO']]

In [73]:
df_export[df_export ['IDMOROSO']=='DNI 26096610      ']

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO
104516,DNI 26096610,DNI,26096610,NaN,TEL.FIJO,PERSONAL,48244551,Propio,AH,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
104517,DNI 26096610,DNI,26096610,NaN,TEL.FIJO,PERSONAL,01148244551,Propio,AH,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
278574,DNI 26096610,DNI,26096610,NaN,TEL.CELULAR,PERSONAL,111554215463,Propio,AH,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
278575,DNI 26096610,DNI,26096610,NaN,TEL.FIJO,PERSONAL,001554215463,Propio,AH,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5
393849,DNI 26096610,DNI,26096610,NaN,TEL.CELULAR,PERSONAL,1554215463,Propio,AH,2020-12-21 17:06:07.420069,False,NaN,NaN,1.5


In [65]:
#df_export.to_csv('D:\MOPC\Enriquecimiento de Datos\Data Cleaning\Active Historico\Exports\DAM_CONTACTABILIDAD\AH_DAM_CONTACTABILIDAD_telefonos.csv',index=False)